In [10]:
# use new database file to plot
import sqlite3
import numpy as np
import matplotlib.pyplot as plt
import folium
import geopandas as gpd
from shapely import wkb
from binascii import unhexlify
from codecs import encode
import pysal

import load_csv

import seaborn as sns
%matplotlib tk

In [2]:
# MONTH
# DAY OF WEEK
# HOUR OF DAY
# IfHoliday
# ROAD CLASS

# highway
# footway
# surface
# lighting
# side wald
# NUM OF LANE
# ONEWAY
# SPEED LIMIT
# RoadFunctionality
# road curvature???

# AADT
# WEATHER --- TEMPORATURE --ICING

# population density??



In [ ]:
file_name = 'data/NY_crash.csv'
file_name = 'data/IOWA_Crash_Data.csv'
NY_crash = load_csv.run(file_name)

,PK_UID,id,osm_id,highway,footway,name,tiger_cfcc,lanes,service,surface,...,ARR_MIN,HOSP_HR,HOSP_MN,CF1,CF2,CF3,FATALS,DRUNK_DR,Geometry,geometry
0,2593,4,2022,motorway,,Barstow Freeway; Mojave Freeway,,2,,,...,99,88,88,0,0,0,1,0,"[ , , �, ,  ,  , l, , �, �, g, , ], �, �, ...",POINT (-116.3656999999745 35.0849305603458)
1,2825,4,2022,motorway,,Barstow Freeway; Mojave Freeway,,2,,,...,99,99,99,0,0,0,1,1,"[ , , �, ,  ,  , �, 3, @, �, , , ], �, =, ...",POINT (-116.3759527804343 35.08348888987937)
2,28385,10,2067,motorway,,,,3,,,...,99,99,99,0,0,0,1,0,"[ , , �, ,  ,  , �, , �, &, �, ^, S, �, �, ...",POINT (-77.48060000038348 38.35029443972219)
3,28421,10,2067,motorway,,,,3,,,...,99,99,99,7,0,0,1,0,"[ , , �, ,  ,  , �, r, -, �, �, [, S, �, \n,...",POINT (-77.43150000036506 38.40458889022928)
4,28643,10,2067,motorway,,,,3,,,...,99,99,99,0,0,0,1,0,"[ , , �, ,  ,  , b, 6, �, �, , ^, S, �, 9, ...",POINT (-77.4706777801853 38.36048333023286)
5,3277,31,3102198,motorway,,Golden State Freeway,,4,,paved,...,99,99,99,0,0,0,1,1,"[ , , �, ,  ,  , g, /, D, m, W, �, ], �, �, ...",POINT (-118.6772111097213 34.55945000025077)
6,3374,33,2048,motorway,,Christopher Columbus Transcontinental Highway,,4,,,...,99,99,99,0,0,0,1,0,"[ , , �, ,  ,  , �, , -, , 7, , ], �, �, ...",POINT (-116.3939888897059 33.81724999973242)
7,3466,33,2048,motorway,,Christopher Columbus Transcontinental Highway,,4,,,...,99,88,88,0,0,0,1,1,"[ , , �, ,  ,  , �, �, R, \t, �, , ], �, �,...",POINT (-116.4303611096212 33.84098056021799)
8,4706,38,3130999,motorway,,Garden Grove Freeway,,3,,,...,99,99,99,0,0,0,1,0,"[ , , �, ,  ,  , �, >, �, U, �, �, ], �, X, ...",POINT (-118.0454305598928 33.77503889004464)
9,2914,64,3158110,secondary,,Alhambra Avenue,,4,,paved,...,99,88,88,0,0,0,1,0,"[ , , �, ,  ,  , @, , �, �, N, �, ], �, �, ...",POINT (-118.1610583300908 34.07851666976741)
